In [35]:
import duckdb
import yaml
import datetime
import openai

In [36]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

openai.api_key  = PARAM['openai_api']
client = openai.OpenAI(api_key = PARAM['openai_api'])

def get_embedding(text, model=PARAM['vector_embedding_model']):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [37]:
con = duckdb.connect()
con.execute("install duckpgq from community;")
con.execute("load duckpgq;")

con.execute("install ducklake;")
con.execute("load ducklake;")

con.execute("install httpfs;")
con.execute("load httpfs;")

con.execute("install postgres;")
con.execute("load postgres;")

In [38]:
con.sql(f"""

CREATE SECRET(

TYPE postgres,

HOST '{PARAM["supabase_postgres_host"]}',

PORT 5432,

DATABASE postgres,

USER '{PARAM["supabase_postgres_user"]}',

PASSWORD '{PARAM["supabase_postgres_password"]}'

);

""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [39]:
con.sql(f"""

CREATE OR REPLACE SECRET secret (

TYPE gcs,

KEY_ID '{PARAM["gcp_access_key"]}',

SECRET '{PARAM["gcp_secret"]}',

REGION 'asia-northeast1'

);

""")



┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [40]:

con.sql(f"""
  ATTACH 'ducklake:postgres:dbname=postgres' AS sixing_ducklake (
      DATA_PATH 'gcs://{PARAM["gcp_bucket_name"]}/ducklake/'
  );
""")

con.sql("""
USE sixing_ducklake;
        """)

In [41]:
con.sql(f"""
    SELECT *
    FROM Disease
    LIMIT 5;
""")

┌──────────┬───────────────────┬───────────┬──────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [42]:
con.sql(f"""
    SHOW TABLES;
""")

┌──────────────┐
│     name     │
│   varchar    │
├──────────────┤
│ Disease      │
│ Drug         │
│ Drug_Disease │
│ Drug_MOA     │
│ MOA          │
│ Trial        │
└──────────────┘

In [43]:
con.sql(f"""
    SELECT *
    FROM Drug_Disease
    LIMIT 5;
""")

┌──────────┬──────────┐
│   from   │    to    │
│ varchar  │ varchar  │
├──────────┼──────────┤
│ C0065023 │ C0014358 │
│ C0065023 │ C0004623 │
│ C3529078 │ C0010481 │
│ C2987431 │ C0003873 │
│ C2934193 │ C0008677 │
└──────────┴──────────┘

In [44]:
con.sql(f"""
    SELECT *
    FROM Drug_MOA
    LIMIT 5;
""")

┌──────────┬─────────────┐
│   from   │     to      │
│ varchar  │   varchar   │
├──────────┼─────────────┤
│ C0541315 │ N0000175076 │
│ C0541315 │ N0000182137 │
│ C0541315 │ N0000182141 │
│ C0541315 │ N0000175624 │
│ C0065023 │ N0000000233 │
└──────────┴─────────────┘

In [45]:
con.sql(f"""
    SELECT Drug.name, Disease.name
    FROM Drug, Drug_Disease, Disease
    WHERE Drug.cui = Drug_Disease.from AND Disease.cui = Drug_Disease.to
    LIMIT 5;
""")

┌─────────────────────────┬─────────────────────────────────┐
│          name           │              name               │
│         varchar         │             varchar             │
├─────────────────────────┼─────────────────────────────────┤
│ fidaxomicin             │ Enterocolitis, Pseudomembranous │
│ fidaxomicin             │ Bacterial Infections            │
│ pasireotide diaspartate │ Cushing Syndrome                │
│ tofacitinib citrate     │ Rheumatoid Arthritis            │
│ olodaterol              │ Bronchitis, Chronic             │
└─────────────────────────┴─────────────────────────────────┘

In [46]:

con.sql("""
CREATE PROPERTY GRAPH drug_graph
  VERTEX TABLES (
    Drug, Disease, MOA
  )
EDGE TABLES (
  Drug_Disease 	SOURCE KEY ('from') REFERENCES Drug (cui)
                DESTINATION KEY ('to') REFERENCES Disease (cui)
  LABEL MAY_TREAT,
  Drug_MOA SOURCE KEY ('from') REFERENCES Drug (cui)
          DESTINATION KEY ('to') REFERENCES MOA (id)
  LABEL HAS_MOA
);
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

In [47]:
con.sql("""SELECT DISTINCT disease_name
  FROM GRAPH_TABLE (drug_graph
  MATCH
  (mo:MOA WHERE mo.name = 'GABA B Receptor Interactions')<-[h:HAS_MOA]-(i:Drug)-[m:MAY_TREAT]->(c:Disease)
  COLUMNS (c.name AS disease_name)
  )
  LIMIT 5;
""")

┌───────────────────────────┐
│       disease_name        │
│          varchar          │
├───────────────────────────┤
│ Bipolar Disorder          │
│ Alzheimer's Disease       │
│ Mood Disorders            │
│ Epilepsy, Complex Partial │
│ Absence Epilepsy          │
└───────────────────────────┘

In [48]:
con.sql(f"""
  SELECT *
  FROM Trial
  WHERE "Study Title" LIKE '%MAXCLARITY%';
""")

┌────────────┬─────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────┬───────────────────────────────┬───────────────────┬─────────┬───────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────┬────────────────────────┬────────────────────────────┬───────────────────────┬────────────────────────┬──────────────────────────────┬─────────────┬─────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────┬───────────────────┬─────────────────────────────────┬─────────────────────────────────────────────────────────────

In [49]:
vector = get_embedding("joint-related pain")

con.sql(f"""
SELECT cui, name
FROM Disease
ORDER BY array_cosine_similarity(definitionEmbedding::FLOAT[1536], {vector}::FLOAT[1536]) DESC
LIMIT 10;
""")

┌──────────┬────────────────────────┐
│   cui    │          name          │
│ varchar  │        varchar         │
├──────────┼────────────────────────┤
│ C0022408 │ Arthropathy            │
│ C0038013 │ Ankylosing spondylitis │
│ C0004604 │ Back Pain              │
│ C0026858 │ Musculoskeletal Pain   │
│ C0949691 │ Spondylarthropathies   │
│ C0024031 │ Low Back Pain          │
│ C0040460 │ Toothache              │
│ C0003864 │ Arthritis              │
│ C0015468 │ Facial Pain            │
│ C0009088 │ Cluster Headache       │
├──────────┴────────────────────────┤
│ 10 rows                 2 columns │
└───────────────────────────────────┘

In [50]:
con.sql(f"""
SELECT * FROM snapshots();
""")

┌─────────────┬───────────────────────────────┬────────────────┬────────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                            changes                             │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                    map(varchar, varchar[])                     │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼────────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2026-01-15 14:13:06.607265+09 │              0 │ {schemas_created=[main]}                                       │ NULL    │ NULL           │ NULL              │
│           1 │ 2026-01-15 14:13:14.141913+09 │              1 │ {tables_created=[main.Drug], tables_inserted_into=

In [51]:
con.close()